In [1]:
import numpy as np
import h5py
from skimage import io
from matplotlib import pyplot as plt
import torch
from torch.autograd import Variable
import torch.optim as optim

In [367]:
fil=h5py.File('shapemodel.h5','r')
fly=open('testface_landmarks.txt')
ftxt=fly.readlines()
for i in range(len(ftxt)):
    ftxt[i]=ftxt[i].strip('\n').split(' ')
for i in range(68):
    for j in range(2):
        ftxt[i][j]=float(ftxt[i][j])

In [368]:
#对sigma和s_mean_raw加一步标准化;
sigma=torch.FloatTensor(fil['sigma'][:])#50*1;
sigma=(sigma-torch.mean(sigma))/torch.std(sigma)
sigma=Variable(sigma)
s_mean_raw=torch.FloatTensor(fil['mean_shape'][:])  #159645*1;
s_mean_raw=(s_mean_raw-torch.mean(s_mean_raw))/torch.std(s_mean_raw)
s_raw=torch.FloatTensor(fil['pca_basis'][:])   #159645*50;
for i in range(50):
    s_raw[:,i]=(s_raw[:,i]-torch.mean(s_raw[:,i]))/torch.std(s_raw[:,i])

In [369]:
#sigma=torch.FloatTensor(fil['sigma'][:])#50*1;
#sigma=Variable(sigma)
#s_mean_raw=torch.FloatTensor(fil['mean_shape'][:])  #159645*1;
#s_raw=torch.FloatTensor(fil['pca_basis'][:])   #159645*50;
U_label=Variable(torch.FloatTensor(np.array(ftxt).T))
d=fil['keypoints'][:][0]
s_mean=torch.rand(204,1)
s=torch.rand((204,50))
for i in range(204):
    if i%3==0:
        s_mean[i]=s_mean_raw[d[i/3]*3]
        s_mean[i+1]=s_mean_raw[d[i/3]*3+1]
        s_mean[i+2]=s_mean_raw[d[i/3]*3+2]
for i in range(50):
    for j in range(204):
        if j%3==0:
            s[j,i]=s_raw[d[j/3]*3,i]
            s[j+1,i]=s_raw[d[j/3]*3+1,i]
            s[j+2,i]=s_raw[d[j/3]*3+2,i]
s=Variable(s)
s_mean=Variable(s_mean)
P=Variable(torch.FloatTensor([[1,0,0],[0,1,0]]))

In [370]:
#f=Variable(torch.rand(1),requires_grad=True)
#alpha_r=np.random.randint(-10,10,(50,1))/10
#换一下随即初始化的方式;
alpha=Variable(torch.Tensor(50,1).uniform_(-1,1),requires_grad=True)
S=s_mean+s.mm(alpha)
S=torch.t(S.view(68,3))
R=Variable(torch.Tensor(3,3).uniform_(-2,2),requires_grad=True)
t=Variable(torch.Tensor(2).uniform_(0,2),requires_grad=True)
U_proj=P.mm(R).mm(S)
for i in range(68):
    U_proj[:,i]=U_proj[:,i]+t

In [371]:
optimizer = optim.SGD([R,t,alpha], lr = 0.001)
#optimizer = optim.Adam([R,t,alpha], lr = 0.000000000001)
#adjust learning_rate;
#lambda1 = lambda epoch: epoch // 30
#lambda2 = lambda epoch: 0.95 ** epoch
#scheduler = optim.LambdaLR(optimizer, lr_lambda=[lambda1, lambda2])

In [372]:
loss=((alpha*alpha)/(sigma*sigma)).mean()+((U_proj-U_label)*(U_proj-U_label)).mean()
loss.backward()
optimizer.step()

In [373]:
loss_print=[]
loss_100=0

In [ ]:
for i in range(500000):
    optimizer.zero_grad()
    alpha=alpha
    S=s_mean+s.mm(alpha)
    S=torch.t(S.view(68,3))
    R=R
    t=t
    U_proj=P.mm(R).mm(S)
    for ii in range(68):
        U_proj[:,ii]=U_proj[:,ii]+t
    loss=((alpha*alpha)/(sigma*sigma)).mean()+((U_proj-U_label)*(U_proj-U_label)).mean()
    loss_100=loss_100+loss.data[0]
    #print;
    loss.backward()
    optimizer.step()
    #scheduler.step()
    if i%100==0:
        #print loss_100
        loss_print.append(loss_100/100.0)
        print(loss_100/100.0)
        loss_100=0
#plt.plot(loss_print[10:])

108.513225784
108.254841766
108.000865707
107.750667191
107.504117661
107.26088501
107.021329041
106.785146637
106.55248909
106.322458878
106.096126099
105.872500076
105.651327133
105.433933945
105.218382797
105.00560112
104.795841827
104.587929153
104.382997971
104.180176239
103.979409637
103.781293793
103.585498352
103.391333694
103.199199524
103.009803543
102.821068878
102.635315933
102.450906296
102.268921814
102.087308502
101.909068375
101.731704407
101.556476135
101.382449341
101.209708252
101.038348618
100.869031525
100.701488571
100.534679642
100.369120026
100.205352936
100.04416832
99.8818289185
99.7224642944
99.5635137939


In [405]:
loss_print[5100:]

[]

In [393]:
#save variable,use np.save(); the variable need to save are:f,R,t,alpha,loss_print,U_proj;
np.save('f.npy',t.data.numpy())
np.save('R.npy',R.data.numpy())
np.save('alpha.npy',alpha.data.numpy())
np.save('loss.npy',np.array(loss_print))
np.save('U.npy',U_proj.data.numpy())

In [402]:
U_label

Variable containing:

Columns 0 to 7 
  83.0361   85.0362   88.4093   91.7430   95.9245  102.4533  109.8493  119.6364
  92.3779  104.4397  115.2020  124.8826  135.3706  143.4386  148.1827  152.0673

Columns 8 to 15 
 133.6377  147.1751  156.1813  162.6417  167.4982  169.5993  170.9633  172.1913
 153.6329  149.2142  143.6258  137.7211  128.7834  117.8099  107.5250   96.0167

Columns 16 to 23 
 172.0603   92.0248   96.4519  102.5672  108.5833  114.1050  136.2818  141.6346
  83.6002   78.0848   71.7372   69.7462   69.8002   71.1668   68.3715   65.8164

Columns 24 to 31 
 147.7133  154.3363  160.0225  126.8195  127.4933  128.1749  128.7587  122.2041
  64.3658   65.2733   71.1240   84.8714   93.0088  100.6941  107.0047  112.7630

Columns 32 to 39 
 125.1339  129.3370  133.3688  136.3219  101.3609  104.6649  110.0171  115.0647
 113.4015  113.9283  112.5248  111.2459   87.3450   84.6894   84.3479   85.8816

Columns 40 to 47 
 110.8103  105.4986  138.3195  142.9577  148.4163  152.2320  148.260

In [401]:
U_proj

Variable containing:

Columns 0 to 7 
  84.3808   83.8318   89.0632   91.3499   96.8994  103.5478  108.2996  119.0298
  92.3605  103.4624  114.8887  125.0970  135.8044  142.2077  147.8119  153.2513

Columns 8 to 15 
 133.4798  146.5857  156.3241  162.2541  168.5002  171.2031  170.2704  170.7020
 152.8125  149.4647  142.9214  138.1297  128.9337  117.1460  108.1419   95.7200

Columns 16 to 23 
 172.8249   90.6426   96.4700  102.7323  108.1348  114.3082  135.3802  142.1728
  83.9752   79.1180   69.7691   69.7686   72.9683   69.9341   65.6015   65.1247

Columns 24 to 31 
 148.1342  153.9646  159.6825  125.5563  126.6205  129.4978  129.9979  120.2220
  66.1203   66.1394   71.9323   86.6281   91.0939  101.3653  106.6194  111.9914

Columns 32 to 39 
 124.2410  128.9195  133.3924  136.9647  102.5897  105.4228  109.8738  114.3573
 113.8283  114.0957  112.8944  111.6448   87.2141   84.4421   84.4840   86.3123

Columns 40 to 47 
 110.4951  106.9495  139.0249  143.2516  147.8823  152.7877  147.627

In [196]:
alpha.data.view((1,50))



Columns 0 to 9 
 0.0793  0.5303  0.9034  0.1460  0.8055  0.6112  0.2066  0.7637  0.6341  0.9574

Columns 10 to 19 
 0.7608  0.7607  0.6103  0.9933  0.8063  0.7225  0.0139  0.2338  0.5233  0.3382

Columns 20 to 29 
 0.2916  0.8532  0.9667  0.9085  0.6751  0.5008  0.7224  0.7926  0.2575  0.2356

Columns 30 to 39 
 0.2784  0.9689  0.6170  0.4036  0.8653  0.0261  0.2742  0.7374  0.4967  0.2771

Columns 40 to 49 
 0.7585  0.2879  0.2400  0.5845  0.0239  0.1285  0.3464  0.0037  0.0412  0.7082
[torch.FloatTensor of size 1x50]

In [155]:
U_label

Variable containing:

Columns 0 to 7 
  83.0361   85.0362   88.4093   91.7430   95.9245  102.4533  109.8493  119.6364
  92.3779  104.4397  115.2020  124.8826  135.3706  143.4386  148.1827  152.0673

Columns 8 to 15 
 133.6377  147.1751  156.1813  162.6417  167.4982  169.5993  170.9633  172.1913
 153.6329  149.2142  143.6258  137.7211  128.7834  117.8099  107.5250   96.0167

Columns 16 to 23 
 172.0603   92.0248   96.4519  102.5672  108.5833  114.1050  136.2818  141.6346
  83.6002   78.0848   71.7372   69.7462   69.8002   71.1668   68.3715   65.8164

Columns 24 to 31 
 147.7133  154.3363  160.0225  126.8195  127.4933  128.1749  128.7587  122.2041
  64.3658   65.2733   71.1240   84.8714   93.0088  100.6941  107.0047  112.7630

Columns 32 to 39 
 125.1339  129.3370  133.3688  136.3219  101.3609  104.6649  110.0171  115.0647
 113.4015  113.9283  112.5248  111.2459   87.3450   84.6894   84.3479   85.8816

Columns 40 to 47 
 110.8103  105.4986  138.3195  142.9577  148.4163  152.2320  148.260